In [3]:
import numpy as np
import pandas as pd
import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from tqdm import tqdm
import datetime
import json
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=True)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [4]:
path = 'C:/Users/user/Desktop/TodayLearn/TiL/Code/Project2/Projcet02/1_smart_card_data'
 
df = pd.read_csv(f'{path}/smartcard_Chain.csv', encoding = 'cp949')


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/TodayLearn/TiL/Code/Project2/Projcet02/1_smart_card_data/smartcard_Chain.csv'

In [3]:
df1 = df.copy()

In [4]:
df1.shape

(8206179, 23)

# 데이터 정제

## 결측치 및 이상치 확인, 외부 데이터 병합

In [5]:
df1['route_id'].replace(np.nan, -1, inplace = True)

In [6]:
df1['Circuity'] = np.where(df1['Circuity'] == 0, np.nan, df1['Circuity'])
df1['Circuity'] = np.where(df1['Haversine'] == 0, np.nan, df1['Circuity'] )

In [7]:
# 시간대 (분단위로 변화)
df1['total_time(m)'] = (df1['total_time'] // 60)

In [8]:
df1.head(5)

,card_name,Bus,Subway,on_area_name,on_lon,on_lat,on_dong,on_hour,on_minute,on_second,off_area_name,off_lon,off_lat,off_dong,off_hour,off_minute,off_second,no_transfer,total_distance,total_time,route_id,Haversine,Circuity,total_time(m)
0,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,영등포시장역,126.905118,37.522694,1119074,12,42,8,충무로역,126.993624,37.561135,1102060,13,16,20,0,11000,2052,-1.0,8897.367673,1.236321,34
1,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,충무로역,126.993624,37.561135,1102060,16,2,9,중곡역,127.084283,37.565891,1105057,16,36,50,0,9400,2081,-1.0,8008.336598,1.173777,34
2,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,중곡역,127.084283,37.565891,1105057,18,22,39,선유도역,126.893294,37.538159,1119062,19,19,46,0,20100,3427,-1.0,17116.783674,1.174286,57
3,+++19/4InCnN/eZXlK0pT7oFlp3qEbVXHX4NFSiyAN8=,True,False,LG전자.문래중,126.885883,37.520390,1119076,16,11,56,문래역,126.894412,37.519063,1119076,16,15,3,0,768,187,11110803.0,766.539891,1.001905,3
4,+++3lFnkOIxAPXhin+oeJBZJeI8l5KOhgxcjhFqGTlc=,True,False,강남구청.강남세무서,127.047293,37.518428,1123078,18,4,57,청담동래미안아파트,127.042912,37.517653,1123078,18,6,30,0,397,93,11110214.0,395.878621,1.002833,1


In [9]:
# 시군구명칭.csv 파일 병합(외부자료)

sample = pd.read_csv(f'{path}/addtional_data/seoul_id.csv', encoding = 'cp949')

In [10]:
sample.rename(columns = { '행정구역번호' : 'on_dong'}, inplace = True)
sample

,on_dong,시군구명칭,읍면동명칭
0,1101053,종로구,사직동
1,1101054,종로구,삼청동
2,1101055,종로구,부암동
3,1101056,종로구,평창동
4,1101057,종로구,무악동
...,...,...,...
420,1125070,강동구,둔촌1동
421,1125071,강동구,둔촌2동
422,1125072,강동구,암사1동
423,1125073,강동구,천호2동


In [11]:
df1 = df1.merge(sample, on = 'on_dong', how = 'left')

In [15]:
df1.dropna(inplace = True)

In [16]:
df1.isnull().sum()

card_name         0
Bus               0
Subway            0
on_area_name      0
on_lon            0
on_lat            0
on_dong           0
on_hour           0
on_minute         0
on_second         0
off_area_name     0
off_lon           0
off_lat           0
off_dong          0
off_hour          0
off_minute        0
off_second        0
no_transfer       0
total_distance    0
total_time        0
route_id          0
Haversine         0
Circuity          0
total_time(m)     0
시군구명칭             0
읍면동명칭             0
dtype: int64

In [17]:
df1.head(5)

,card_name,Bus,Subway,on_area_name,on_lon,on_lat,on_dong,on_hour,on_minute,on_second,off_area_name,off_lon,off_lat,off_dong,off_hour,off_minute,off_second,no_transfer,total_distance,total_time,route_id,Haversine,Circuity,total_time(m),시군구명칭,읍면동명칭
0,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,영등포시장역,126.905118,37.522694,1119074,12,42,8,충무로역,126.993624,37.561135,1102060,13,16,20,0,11000,2052,-1.0,8897.367673,1.236321,34,영등포구,영등포동
1,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,충무로역,126.993624,37.561135,1102060,16,2,9,중곡역,127.084283,37.565891,1105057,16,36,50,0,9400,2081,-1.0,8008.336598,1.173777,34,중구,을지로동
2,+++0VxU0B9DrLE4nxWaOzEArKFNI3ep9kMPVTZVHGNM=,False,True,중곡역,127.084283,37.565891,1105057,18,22,39,선유도역,126.893294,37.538159,1119062,19,19,46,0,20100,3427,-1.0,17116.783674,1.174286,57,광진구,중곡3동
3,+++19/4InCnN/eZXlK0pT7oFlp3qEbVXHX4NFSiyAN8=,True,False,LG전자.문래중,126.885883,37.520390,1119076,16,11,56,문래역,126.894412,37.519063,1119076,16,15,3,0,768,187,11110803.0,766.539891,1.001905,3,영등포구,문래동
4,+++3lFnkOIxAPXhin+oeJBZJeI8l5KOhgxcjhFqGTlc=,True,False,강남구청.강남세무서,127.047293,37.518428,1123078,18,4,57,청담동래미안아파트,127.042912,37.517653,1123078,18,6,30,0,397,93,11110214.0,395.878621,1.002833,1,강남구,청담동
